In [6]:
import qutip as qt
import cvxpy as cp
import numpy as np

In [9]:
# Defining the states

rho = qt.ket2dm(qt.basis(2, 0))
plus = (qt.basis(2, 0) + qt.basis(2, 1)).unit()
sigma = qt.ket2dm(plus)

rho_np = rho.full()
sigma_np = sigma.full()

In [10]:
rho

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[1. 0.]
 [0. 0.]]

In [11]:
sigma

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]

In [12]:
J = cp.Variable((4, 4), complex=True)
W = cp.Variable((2, 2), complex=True)

In [13]:
rho_T = rho_np.T
I = np.eye(2)
temp = cp.kron(I, rho_T) @ J

In [ ]:
# block summation for phi(\rho)

Phi_rho_blocks = []
for x in range(2):
    blk = temp[2*x:2*(x+1), 2*x:2*(x+1)]
    Phi_rho_blocks.append(blk)
Phi_rho = sum(Phi_rho_blocks)


In [16]:
TrY = 0
for y in range(2):
    TrY += J[y*2:(y+1)*2, y*2:(y+1)*2]
constraints = [J >> 0, TrY == np.eye(2)]

In [18]:
# entanglement witness and block matrix must be psd

block = cp.bmat([
    [Phi_rho, W],
    [W.H, sigma_np]
])
constraints += [block >> 0]

In [19]:
objective = cp.Maximize(cp.real(cp.trace(W)))
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.SCS, verbose=True)

(CVXPY) Nov 03 02:27:37 PM: Your problem has 20 variables, 52 constraints, and 0 parameters.
(CVXPY) Nov 03 02:27:37 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 03 02:27:37 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 03 02:27:37 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 03 02:27:37 PM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Nov 03 02:27:37 PM: Compiling problem (target solver=SCS).
(CVXPY) Nov 03 02:27:37 PM: Reduction chain: Complex2Real -> FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCS
(CVXPY) Nov 03 02:27:37 PM: Applying reduction Complex2Real
(CVXPY) Nov 03 02:27:37 PM: Applying reduction FlipObjective
(CVXPY) Nov 03 02:27:37 PM: Applying reduction Dcp2Cone
(CVXPY) Nov 03 02:27:37 PM: Applying reduction CvxAttr2Constr
(CVXPY) Nov 03 02:27:37 PM: App

                                     CVXPY                                     
                                     v1.7.2                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
------------------------------------------------------------------
	       SCS v3.2.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 40, constraints m: 116
cones: 	  z: primal zero / dual free vars: 8
	  s: psd vars: 108, ssize: 3
settings: 

0.7071070986051053

In [20]:
print(f"Optimal fidelity = {prob.value:.6f}")
print("Φ(ρ) =\n", Phi_rho.value)
print("Choi matrix J(Φ) =\n", J.value)

Optimal fidelity = 0.707107
Φ(ρ) =
 [[ 1.00000001e+00+0.j -5.30428618e-11+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j]]
Choi matrix J(Φ) =
 [[ 5.00000003e-01+0.j  5.53504688e-10+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [-5.80045622e-10+0.j  5.00000000e-01+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  5.00000003e-01+0.j
  -6.06547550e-10+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  5.79989957e-10+0.j
   5.00000000e-01+0.j]]


In [22]:
# Extract Kraus ops

J_opt = qt.Qobj(J.value, dims=[[2, 2], [2, 2]])

evals, evecs = J_opt.eigenstates()
kraus_ops = []

for i, val in enumerate(evals):
    if val > 1e-10:  # skip numerical noise
        # Each eigenvector reshaped into a Kraus operator
        vec = evecs[i].full().reshape(2, 2, order='F')
        K = np.sqrt(val) * vec
        kraus_ops.append(qt.Qobj(K))
        print(f"Kraus operator {i}:\n", qt.Qobj(K))

Kraus operator 0:
 Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[-0.13279441  0.        ]
 [ 0.69452548  0.        ]]
Kraus operator 1:
 Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[0.         0.14554132]
 [0.         0.69196656]]
Kraus operator 2:
 Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[-0.          0.69322544]
 [-0.          0.139422  ]]
Kraus operator 3:
 Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.69332589 -0.        ]
 [-0.13892162 -0.        ]]


In [24]:
sum_check = sum([K.dag() * K for K in kraus_ops])
print("\nΣ K†K =\n", sum_check)


Σ K†K =
 Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1. 0.]
 [0. 1.]]


In [ ]:
print("Trace-preserving:", np.allclose(sum_check.full(), np.eye(2)))

Trace-preserving: True


In [31]:
from scipy.linalg import logm

Phi_super = sum(np.kron(K.full(), K.full().conj()) for K in kraus_ops)
L_super = logm(Phi_super)
L = qt.Qobj(L_super, dims=[[2, 2], [2, 2]])


/Users/tchatterjee/anaconda3/envs/pmp/lib/python3.13/site-packages/scipy/linalg/_matfuncs.py:202: LogmExactlySingularWarning: The logm input matrix is exactly singular.
  F = scipy.linalg._matfuncs_inv_ssq._logm(A)


In [32]:
L

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=False
Qobj data =
[[   -2.84049697  -1.57067278j     0.          +0.j
      0.          +0.j             2.84094385  +1.57091988j]
 [ 2285.27065838-177.82283219j   -46.05170186  +0.j
      0.          +0.j         -2285.22566684+177.85080748j]
 [ 2285.27065838-177.82283219j     0.          +0.j
    -46.05170186  +0.j         -2285.22566684+177.85080748j]
 [    2.84049698  +1.57067278j     0.          +0.j
      0.          +0.j            -2.84094384  -1.57091988j]]